In [22]:

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [23]:
import os
import sys
print(os.getcwd())
os.environ.update(os.environ)
        # Add a new environment variable to the operating system
os.environ["RAMP_HOME"] = os.getcwd()
# Print the environment variables to verify that the new variable was added
print(os.environ["RAMP_HOME"])
# sys.path.append('../')
sys.path.append('ramp-code/')


/Users/azanchetta/fAIr_GPU
/Users/azanchetta/fAIr_GPU


In [24]:
import cv2

In [25]:
os.environ["SM_FRAMEWORK"] = "tf.keras"
import ramp.utils
import hot_fair_utilities
# base_path = f"{os.getcwd()}/ramp-data/sample_2"
base_path = "/Users/azanchetta/fAIr-utilities/ramp-data/1_Zanzibar"

In [26]:
from hot_fair_utilities import preprocess
model_input_image_path = f"{base_path}/input"
preprocess_output=f"{base_path}/preprocessed"
preprocess(
            input_path = model_input_image_path,
            output_path = preprocess_output,
            rasterize=True,
            rasterize_options=["binary"],
            georeference_images=True,
        )

Georeferencing for input: 100%|██████████| 963/963 [00:53<00:00, 18.09it/s]
ERROR:fiona._env:PROJ: proj_create_from_database: /usr/local/anaconda3/envs/fairgpu/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
Clipping labels for input: 100%|██████████| 963/963 [02:46<00:00,  5.79it/s]


In [27]:
from hot_fair_utilities import train

In [40]:
train_output = f"{base_path}/train"
final_accuracy, final_model_path = train(
    input_path=preprocess_output,
    output_path=train_output,
    epoch_size=2,
    batch_size=2,
    model="ramp",
    model_home=os.environ["RAMP_HOME"]
)

SyntaxError: invalid character '–' (U+2013) (1311954738.py, line 8)

# here a tf file is created (weights + structure)

In [35]:
print(final_accuracy,final_model_path)

98.1 /Users/azanchetta/fAIr-utilities/ramp-data/1_Zanzibar/train/model-checkpts/20231116-143824/model_20231116-143824_002_0.981.tf


In [36]:
from hot_fair_utilities import predict
prediction_output = f"{base_path}/prediction/output"
predict(
    checkpoint_path=final_model_path,
    input_path=f"{base_path}/prediction/input",
    prediction_path=prediction_output,
)

Using : /Users/azanchetta/fAIr-utilities/ramp-data/1_Zanzibar/train/model-checkpts/20231116-143824/model_20231116-143824_002_0.981.tf
It took 12 sec to load model


2023-11-16 15:02:13.414248: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 5s 5s/step
It took 5 sec to predict with 0.5 Confidence Threshold


Georeferencing for output: 100%|██████████| 3/3 [00:00<00:00, 22.64it/s]

It took 0 sec to georeference


In [38]:
from hot_fair_utilities import polygonize
geojson_output= f"{prediction_output}/prediction.geojson"
polygonize(
    input_path=prediction_output, 
    output_path=geojson_output,
    remove_inputs = True,
)

100%|██████████| 3/3 [00:00<00:00, 200.91mask/s]
ERROR:fiona._env:PROJ: proj_create_from_database: /usr/local/anaconda3/envs/fairgpu/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
Merging components: 100%|██████████| 16/16 [00:02<00:00,  7.92component/s]
